##### Copyright 2020 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Operators and observables

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/operators_and_observables"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/operators_and_observables.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/operators_and_observables.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/operators_and_observables.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq
    
import numpy as np

This guide is directed at those already familiar with quantum operations and observables who want to know how to use them in Cirq.

<table>
<thead>
  <tr>
    <th></th>
    <th>Item</th>
    <th>Object or Example</th>
    <th>Dedicated Guide(s)</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td rowspan="3">Operators</td>
    <td>Unitary operators</td>
    <td>`cirq.X`</td>
    <td>[Gates](gates.ipynb) and [Custom gates](custom_gates.ipynb)</td>
  </tr>
  <tr>
    <td>Measurements</td>
    <td>`cirq.MeasurementGate`</td>
    <td>[Gates](gates.ipynb)</td>
  </tr>
  <tr>
    <td>Noisy channels</td>
    <td>`cirq.DepolarizingChannel`</td>
    <td>[Noise](noise.ipynb)</td>
  </tr>
  <tr>
    <td rowspan="2">Observables</td>
    <td>Pauli strings</td>
    <td>`cirq.PauliString`</td>
    <td></td>
  </tr>
  <tr>
    <td>Hamiltonians</td>
    <td>`cirq.PauliSum`</td>
    <td></td>
  </tr>
</tbody>
</table>

## Operators

Quantum operations (or just *operators*) include unitary gates, measurements, and noisy channels. Every operator in Cirq is an instance of a `cirq.Operation` which supports the Kraus operator representation

$$
\rho \mapsto \sum_{k} A_k \rho A_k^\dagger
$$

where $\sum_{k} A_k^\dagger A_k = I$ and $\rho$ is a quantum state. Operators are defined in the `cirq.ops` module.

### Unitary operators

Standard unitary operators used in quantum information can be found in `cirq.ops`, for example Pauli-$X$ as shown below.

In [ ]:
qbit = cirq.LineQubit(0)
unitary_operation = cirq.ops.X.on(qbit)  # cirq.X can also be used for cirq.ops.X
print(unitary_operation)

Cirq makes a distinction between gates (independent of qubits) and operations (gates acting on qubits). Thus `cirq.X` is a gate where `cirq.X.on(qbit)` is an operation. See the [guide on gates](gates.ipynb) for more details and additional common unitaries defined in Cirq.

> **Note**: The method `cirq.X.on_each` is a utility to apply `cirq.X` to multiple qubits. Similarly for other operations.

Every `cirq.Operation` supports the `cirq.channel` protocol which returns its Kraus operators. (Read more about [protocols](protocols.ipynb) in Cirq.)

In [ ]:
kraus_ops = cirq.channel(unitary_operation)
print(f"Kraus operators of {unitary_operation.gate} are:", *kraus_ops, sep="\n")

Unitary operators also support the `cirq.unitary` protocol.

In [ ]:
unitary = cirq.unitary(cirq.ops.X)
print(f"Unitary of {unitary_operation.gate} is:\n", unitary)

Unitary gates can be raised to powers, for example to implement a $\sqrt{X}$ operation.

In [ ]:
sqrt_not = cirq.X ** (1 / 2)
print(cirq.unitary(sqrt_not))

Any gate can be controlled via `cirq.ControlledGate` as follows.

In [ ]:
controlled_hadamard = cirq.ControlledGate(sub_gate=cirq.H, num_controls=1)
print(cirq.unitary(controlled_hadamard).round(3))

Custom gates can be defined as described in [this guide](custom_gates.ipynb). Some common subroutines which consist of several operations are pre-defined - e.g., `cirq.qft` returns the operations to implement the quantum Fourier transform.

### Measurements

Cirq supports measurements in the computational basis.

In [ ]:
measurement = cirq.MeasurementGate(num_qubits=1, key="key")
print("Measurement:", measurement)

The `key` can be used to identify results of measurements when simulating circuits. A measurement gate acting on a qubit forms an operation.

In [ ]:
measurement_operation = measurement.on(qbit)
print(measurement_operation)

> **Note**: The function `cirq.measure` is a utility to measure a single qubit, and the function `cirq.measure_each` is a utility to measure multiple qubits.

Again measurements are instances of `cirq.Operation`s so the `cirq.channel` protocol can be used to get the Kraus operators.

In [ ]:
kraus_ops = cirq.channel(measurement)
print(f"Kraus operators of {measurement} are:", *kraus_ops, sep="\n\n")

The functions `cirq.measure_state_vector` and `cirq.measure_density_matrix` can be used to perform computational basis measurements on state vectors and density matrices, respectively, represented by NumPy arrays.

In [ ]:
psi = np.ones(shape=(2,)) / np.sqrt(2)
print("Wavefunction:\n", psi.round(3))

In [ ]:
results, psi_prime = cirq.measure_state_vector(psi, indices=[0])

print("Measured:", results[0])
print("Resultant state:\n", psi_prime)

In [ ]:
rho = np.ones(shape=(2, 2)) / 2.0
print("State:\n", rho)

In [ ]:
measurements, rho_prime = cirq.measure_density_matrix(rho, indices=[0])

print("Measured:", measurements[0])
print("Resultant state:\n", rho_prime)

These functions do not modify the input state (`psi` or `rho`) unless the optional argument `out` is provided as the input state.

### Noisy channels

Like common unitary gates, Cirq defines many common noisy channels, for example the depolarizing channel below.

In [ ]:
depo_channel = cirq.DepolarizingChannel(p=0.01, n_qubits=1)
print(depo_channel)

Just like unitary gates and measurements, noisy channels are `cirq.Operation`s, and we can always use `cirq.channel` to get the Kraus operators.

In [ ]:
kraus_ops = cirq.channel(depo_channel)
print(f"Kraus operators of {depo_channel} are:", *[op.round(2) for op in kraus_ops], sep="\n\n")

Some channels can be written

$$
\rho \mapsto \sum_k p_k U_k \rho U_k ^\dagger
$$

where real numbers $p_k$ form a probability distribution and $U_k$ are unitary. Such a *probabilistic mixture* of unitaries supports the `cirq.mixture` protocol which returns $p_k$ and $U_k$. An example is shown below for the bit-flip channel $\rho \mapsto (1 - p) \rho + p X \rho X$.

In [ ]:
bit_flip = cirq.bit_flip(p=0.05)
probs, unitaries = cirq.mixture(bit_flip)

for prob, unitary in cirq.mixture(bit_flip):
    print(f"With probability {prob}, apply \n{unitary}\n")

> **Note**: Any unitary gate/operation supports `cirq.mixture` because it can be interpreted as applying a single unitary with probability one.

Custom noisy channels can be defined as described in [this guide](noise.ipynb).

### In circuits

Any `cirq.Operation` (pre-defined or user-defined) can be placed in a `cirq.Circuit`. An example with a unitary, noisy channel, and measurement is shown below.

In [ ]:
circuit = cirq.Circuit(
    cirq.H(qbit),
    cirq.depolarize(p=0.01).on(qbit),
    cirq.measure(qbit)
)
print(circuit)

The general input to the circuit constructor is a `cirq.OP_TREE`, i.e., an operation or nested collection of operations. Circuits can be manipulated as described in the [circuits guide](circuits.ipynb) and simulated as described in the [simulation guide](simulation.ipynb).

## Observables

Cirq supports observables which are Pauli strings or linear combinations of Pauli strings. Such objects can be used to compute expectation values.

### Pauli strings

Pauli products or strings are supported via `cirq.PauliString`. For example, the Pauli $Z_0 Z_1$ (where subscripts denote qubit indices) can be represented as follows.

In [ ]:
# Qubit register
qreg = cirq.NamedQubit.range(2, prefix="q")

# PauliString Z_0 Z_1
zz = cirq.PauliString(cirq.Z(q) for q in qreg)
print(zz)

The matrix of a Pauli string can be returned via:

In [ ]:
zz.matrix()

and Pauli strings can have arbitrary coefficients.

In [ ]:
new = (1.0 - 0.1j) * zz
print(new)

### Pauli sums

A `cirq.PauliSum` is a linear combination of `cirq.PauliString`s and represents a Hamiltonian (or general observable) in the Pauli basis. To represent the Hamiltonian $H = 1.5 ZZ - 0.7 XX$, we can first define the $XX$ Pauli string:

In [ ]:
xx = cirq.PauliString(cirq.X(q) for q in qreg)
print(xx)

Then form the linear combination.

In [ ]:
psum = 1.5 * zz - 0.7 * xx
print(psum)

Like Pauli strings, we can get the matrix of a `cirq.PauliSum`:

In [ ]:
psum.matrix()

### Expectation values

Given a state $\rho$ that is prepared by a circuit, expectation values $\text{Tr} [ \rho H ]$ where $H$ is an observable can be computed as follows. First, an example circuit:

In [ ]:
circuit = cirq.Circuit(cirq.ops.H.on_each(qreg))
print(circuit)

The pattern for computing $\text{Tr} [ \rho H ]$ is as follows.

In [ ]:
# Define a PauliSumCollector.
collector = cirq.PauliSumCollector(circuit, psum, samples_per_term=10_000)

# Provide a sampler.
collector.collect(sampler=cirq.Simulator())

# Estimate the observable.
energy = collector.estimated_energy()
print("Energy:", energy)

Note that this method uses sampling with a number of samples given by `samples_per_term`. 

Expectation values can also be computed from NumPy array representations of $\rho$. For example, given a state vector we can do the following.

In [ ]:
# Get the state vector.
psi = circuit.final_state_vector()

# Compute the expectation value.
energy = psum.expectation_from_state_vector(
    state_vector=psi, qubit_map={q: i for i, q in enumerate(qreg)}
)
print("Energy:", energy.real)

And given a density matrix, we can compute the expectation via:

In [ ]:
# Get the density matrix.
dsim = cirq.DensityMatrixSimulator()
rho = dsim.simulate(circuit).final_density_matrix

# Compute the expectation value.
energy = psum.expectation_from_density_matrix(
    state=rho, qubit_map={q: i for i, q in enumerate(qreg)}
)
print("Energy:", energy.real)

### Pauli expansions

The function `cirq.pauli_expansion` can return Pauli basis representations of certain objects. For example, given the circuit $H \otimes H$ from above:

In [ ]:
circuit

The Pauli expansion can be obtained via:

In [ ]:
psum = cirq.pauli_expansion(circuit)
print(psum)

Generally, the argument to `cirq.pauli_expansion` must support the `_pauli_expansion_` method or have a small unitary representation. The argument `default` can be provided to to return a default value if no expansion can be computed.